# Sunspots – T_log v0.5 – Phase 2 (Notebook principal)

Ce notebook implémente la **Phase 2** du pipeline T_log v0.5 pour les taches solaires (sunspots).

- Les **notes détaillées** (objectif, formules, organisation globale) sont dans :  
  [THEORY/SunspotPhase2Tlog_notes.md](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/THEORY/SunspotPhase2Tlog_notes.md:0:0-0:0)
- Les **dépendances Python** sont listées dans :  
  [requirements.txt](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/requirements.txt:0:0-0:0)  
  (sans versions fixées, pour éviter les conflits d’environnement).

## Règles de ce notebook

- Organisation stricte **Markdown → Code → Markdown → Code ...**
- Toutes les **actions sur les fichiers** doivent rester à l’intérieur du dossier :  
  `c:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog`
- Aucun fichier ne doit être créé ou modifié dans les dossiers de la **Phase 1**.

## Blocs de la Phase 2 (vue d’ensemble)

- **Bloc 0** – Infrastructure Phase 2 (racine, dossiers, logging, manifeste, requirements)
- **Bloc 1** – Données & traçabilité
- **Bloc 2** – Prétraitements & fenêtres
- **Bloc 3** – Estimation de la dimension effective `d` (multi-méthodes)
- **Bloc 4** – Calcul de \( T_{\log}(n, d) \) avec incertitude
- **Bloc 5** – Baselines & modèles nuls
- **Bloc 6** – Falsification & analyses de sensibilité
- **Bloc 7** – Synthèse & rapport Phase 2

Dans la suite, nous commencerons par le **Bloc 0**, en définissant la structure des dossiers, le mécanisme de logging et le manifeste de run, toujours de façon incrémentale (une étape à la fois).

In [1]:
# ============================================================
# ⚙️ Installation des dépendances du projet
# Cette cellule garantit que toutes les librairies nécessaires sont installées.
# ============================================================

import subprocess
import sys

def install_requirements(file_path="requirements.txt"):
    """Installe les paquets listés dans requirements.txt."""
    print(f"Installation/Mise à jour des dépendances via {file_path}...")
    try:
        # Exécute la commande pip
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", file_path])
        print("\n✅ Toutes les dépendances ont été installées ou mises à jour avec succès.")
    except subprocess.CalledProcessError as e:
        print(f"\n❌ ERREUR lors de l'installation des dépendances : {e}")

# Exécuter l'installation
install_requirements()


Installation/Mise à jour des dépendances via requirements.txt...

✅ Toutes les dépendances ont été installées ou mises à jour avec succès.


## Bloc 0 – Infrastructure Phase 2

Dans ce bloc, nous définissons l’infrastructure de base du projet Sunspots – T_log v0.5 – Phase 2 :

- fixer la **racine du projet Phase 2** au dossier : `SunspotPhase2Tlog` ;
- créer (ou vérifier) les sous-dossiers internes au projet :
  - [logs/](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/CLEANvs%20NOCLEAN/Clean/v1/V2OrPipeline%E2%80%917_%28PUHM%29/logs:0:0-0:0), `logs/runs/`
  - `reports/`
  - `artifacts/`
  - `cache/`
  - `config/`
  - `data_phase2/` (optionnel : copie locale ou extraits des données Sunspots Phase 1)
- initialiser un **RUN_ID**, une **graine globale** (`GLOBAL_SEED`) et un **manifeste de run** (fichier JSON) pour l’audit.

Règle importante :

- Aucun dossier/fichier ne doit être créé **en dehors** de :
  `c:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog`

La cellule de code suivante va :

1. fixer `PHASE2_ROOT` sur le dossier qui contient ce notebook (`SunspotPhase2Tlog`) ;
2. vérifier que nous sommes bien dans ce dossier (sinon : erreur explicite) ;
3. créer les sous-dossiers listés ci-dessus (avec `exist_ok=True`) ;
4. créer un dossier de run unique sous `logs/runs/` et y écrire un `manifest.json` minimal ;
5. afficher un résumé en sortie de cellule (chemins, RUN_ID, statut du manifeste).

In [2]:
# Bloc 0.1 – Infrastructure Phase 2 : racine, dossiers, manifeste de run

from pathlib import Path
from datetime import datetime
import json
import platform
import sys
import uuid
import random

import numpy as np

# 0.1.1 – Définition de la racine Phase 2
PHASE2_ROOT = Path().resolve()
print(f"PHASE2_ROOT détecté : {PHASE2_ROOT}")

# Vérification : on impose que le notebook soit dans 'SunspotPhase2Tlog'
if PHASE2_ROOT.name != "SunspotPhase2Tlog":
    raise RuntimeError(
        "Ce notebook doit être exécuté depuis le dossier 'SunspotPhase2Tlog'. "
        f"Dossier actuel : {PHASE2_ROOT}"
    )

# 0.1.2 – Graine globale
GLOBAL_SEED = 42
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

# 0.1.3 – Sous-dossiers internes au projet Phase 2
subdirs = [
    "logs",
    "logs/runs",
    "reports",
    "artifacts",
    "cache",
    "config",
    "data_phase2",
]

created_dirs = []
for sd in subdirs:
    full_path = PHASE2_ROOT / sd
    full_path.mkdir(parents=True, exist_ok=True)
    created_dirs.append(str(full_path))

# 0.1.4 – RUN_ID et dossier de run
timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
short_uid = str(uuid.uuid4())[:8]
RUN_ID = f"sunspots_phase2_{timestamp_str}_{short_uid}"

RUN_DIR = PHASE2_ROOT / "logs" / "runs" / RUN_ID
RUN_DIR.mkdir(parents=True, exist_ok=True)

# 0.1.5 – Manifeste minimal de run
manifest = {
    "run_id": RUN_ID,
    "created_at": datetime.now().isoformat(),
    "phase2_root": str(PHASE2_ROOT),
    "notebook_name": "SunspotPhase2Tlog.ipynb",
    "python_version": sys.version,
    "platform": platform.platform(),
    "global_seed": GLOBAL_SEED,
}

manifest_path = RUN_DIR / "manifest.json"

try:
    with open(manifest_path, "w", encoding="utf-8") as f:
        json.dump(manifest, f, indent=2)
    manifest_status = "OK"
except Exception as e:
    manifest_status = f"ERROR: {e}"

# 0.1.6 – Résumé en sortie de cellule (pour audit)
print("\n=== Sunspots – T_log v0.5 – Phase 2 ===")
print("Bloc 0.1 – Infrastructure Phase 2 (racine, dossiers, manifeste)")
print(f"GLOBAL_SEED     : {GLOBAL_SEED}")
print(f"RUN_ID          : {RUN_ID}")
print("\nSous-dossiers créés/vérifiés :")
for d in created_dirs:
    print(f"  - {d}")
print(f"\nManifeste de run : {manifest_path} [{manifest_status}]")

PHASE2_ROOT détecté : C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog

=== Sunspots – T_log v0.5 – Phase 2 ===
Bloc 0.1 – Infrastructure Phase 2 (racine, dossiers, manifeste)
GLOBAL_SEED     : 42
RUN_ID          : sunspots_phase2_20251117_165223_06c8e057

Sous-dossiers créés/vérifiés :
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\logs
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\logs\runs
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\reports
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\artifacts
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\cache
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\config
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2

Manifeste de run : C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\logs\runs\sunspots_phase2_20251117_165223_06c8e057\m

### Bloc 0.2 – Mise en place du logging (sans horodatage)

Le Bloc 0.1 a :

- fixé la racine du projet Phase 2 à :  
  `C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog`
- créé/vérifié les sous-dossiers internes (`logs/`, `logs/runs/`, `reports/`, `artifacts/`, `cache/`, `config/`, `data_phase2/`) ;
- créé un `RUN_ID` unique et un dossier de run associé sous `logs/runs/<RUN_ID>/` ;
- écrit un fichier `manifest.json` minimal.

Pour l’audit, nous avons besoin d’un système de logging, mais **sans horodatage** dans les messages
pour préserver la reproductibilité (les mêmes cellules doivent produire les mêmes logs si on les
ré-exécute dans les mêmes conditions).

Nous allons donc définir :

- une fonction `log_message(...)` qui :
  - utilise un **compteur d’étapes** (`LOG_STEP`) ;
  - écrit des lignes de la forme :  
    `[STEP=1][INFO][BLOC_0.2] message...`
  - affiche ce texte dans la sortie du notebook ;
  - et l’ajoute dans un fichier `run_log.txt` sous `logs/runs/<RUN_ID>/`.
- une fonction `log_metric(...)` qui enregistre des valeurs importantes dans un fichier
  `metrics.jsonl` (format JSONL), en associant chaque enregistrement à une étape (step)
  et au `RUN_ID`.

Ce système de logging doit :

- rester **déterministe** (pas de date/heure dans les messages) ;
- être **local** au projet Phase 2 (tous les fichiers dans `SunspotPhase2Tlog`) ;
- pouvoir être exécuté **sur une autre machine** (PC local, Google Colab) sans modification.

In [3]:
# Bloc 0.2 – Logging : fonctions utilitaires et fichiers de log (sans horodatage)

import json

# Vérification : RUN_DIR doit venir du Bloc 0.1
try:
    RUN_DIR
    RUN_ID
except NameError:
    raise RuntimeError(
        "RUN_DIR ou RUN_ID n'est pas défini. "
        "Assurez-vous d'avoir exécuté le Bloc 0.1 (infrastructure) avant ce bloc."
    )

# 0.2.1 – Chemins des fichiers de log pour ce run
LOG_TEXT_PATH = RUN_DIR / "run_log.txt"
LOG_JSONL_PATH = RUN_DIR / "metrics.jsonl"

print(f"Fichier de log texte   : {LOG_TEXT_PATH}")
print(f"Fichier de log metrics : {LOG_JSONL_PATH}")

# 0.2.2 – Compteur d'étapes global pour les logs
LOG_STEP = 0

def log_message(level, message, block=None):
    """
    Loggue un message texte avec un compteur d'étapes (LOG_STEP), sans horodatage.
    - Affiche en sortie de cellule.
    - Ajoute la même ligne dans le fichier run_log.txt.
    """
    global LOG_STEP
    LOG_STEP += 1

    prefix = f"[STEP={LOG_STEP}][{level.upper()}]"
    if block is not None:
        prefix += f"[{block}]"
    line = f"{prefix} {message}"

    # Affichage dans la sortie du notebook
    print(line)

    # Écriture dans le fichier texte
    with open(LOG_TEXT_PATH, "a", encoding="utf-8") as f:
        f.write(line + "\n")


def log_metric(name, value, step=None, extra=None):
    """
    Enregistre une métrique / valeur importante dans un fichier JSONL (metrics.jsonl).

    - name  : nom de la métrique (str)
    - value : valeur de la métrique (numérique ou autre sérialisable en JSON)
    - step  : étape / index facultatif (par défaut : LOG_STEP courant)
    - extra : dict optionnel avec des métadonnées supplémentaires
    """
    if step is None:
        step_val = LOG_STEP
    else:
        step_val = step

    record = {
        "run_id": RUN_ID,
        "step": step_val,
        "metric": name,
        "value": value,
    }
    if extra is not None:
        record["extra"] = extra

    with open(LOG_JSONL_PATH, "a", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

    # Feedback minimal en sortie de cellule
    print(f"[METRIC][{name}] = {value} (step={step_val})")


# 0.2.3 – Petit test de logging pour vérifier le système
log_message("INFO", "Initialisation du système de logging terminée.", block="BLOC_0.2")
log_metric("test_metric_logging", 1.0, extra={"note": "test initial"})

Fichier de log texte   : C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\logs\runs\sunspots_phase2_20251117_165223_06c8e057\run_log.txt
Fichier de log metrics : C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\logs\runs\sunspots_phase2_20251117_165223_06c8e057\metrics.jsonl
[STEP=1][INFO][BLOC_0.2] Initialisation du système de logging terminée.
[METRIC][test_metric_logging] = 1.0 (step=1)


## Bloc 1.0 – Détection de la source de données Sunspots (zip local)

Dans ce projet Phase 2, la source **canonique** des données Sunspots est le fichier zip
localisé dans le même dossier que ce notebook :

- `sunspots.zip` (dans `SunspotPhase2Tlog/`)

Ce zip est supposé être **identique** à celui disponible sur Kaggle, mais :

- la Phase 2 ne dépend **pas** d’un accès réseau vers Kaggle ;
- tout se fait à partir du fichier `sunspots.zip` déjà présent dans le projet.

Contraintes :

- aucun chemin absolu spécifique à une machine ne doit être codé en dur ;
- le notebook doit rester exécutable sur une autre machine (ou sur Google Colab)
  tant que la structure du projet est respectée et que `sunspots.zip` est présent
  dans `SunspotPhase2Tlog/`.

Dans ce Bloc 1.0, nous allons simplement :

1. définir le chemin du zip comme `PHASE2_ROOT / "sunspots.zip"` ;
2. vérifier que ce fichier existe ;
3. logguer cette information pour l’audit.

L’extraction du contenu du zip (création de fichiers CSV dans `data_phase2/`) sera faite
dans un **Bloc 1.1** séparé.

In [4]:
# Bloc 1.0 – Détection de la source de données Sunspots (zip local)

from pathlib import Path

# 1.0.1 – Chemin du zip Sunspots (relatif à la racine Phase 2)
SUNSPOTS_ZIP_PATH = PHASE2_ROOT / "sunspots.zip"

print(f"PHASE2_ROOT        : {PHASE2_ROOT}")
print(f"SUNSPOTS_ZIP_PATH  : {SUNSPOTS_ZIP_PATH}")

# 1.0.2 – Vérification d'existence
if not SUNSPOTS_ZIP_PATH.exists():
    raise FileNotFoundError(
        f"Le fichier sunspots.zip est introuvable à l'emplacement attendu : {SUNSPOTS_ZIP_PATH}.\n"
        "Assurez-vous que le zip Sunspots (identique à celui de Kaggle) est placé dans le dossier "
        "SunspotPhase2Tlog, à côté du notebook."
    )

# 1.0.3 – Logging pour audit
log_message(
    "INFO",
    f"Fichier sunspots.zip détecté à l'emplacement : {SUNSPOTS_ZIP_PATH}",
    block="BLOC_1.0",
)
log_metric(
    "sunspots_zip_present",
    True,
    extra={"zip_path": str(SUNSPOTS_ZIP_PATH.name)},
)

PHASE2_ROOT        : C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog
SUNSPOTS_ZIP_PATH  : C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\sunspots.zip
[STEP=2][INFO][BLOC_1.0] Fichier sunspots.zip détecté à l'emplacement : C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\sunspots.zip
[METRIC][sunspots_zip_present] = True (step=2)


### Bloc 1.1 – Inspection et extraction de `sunspots.zip` dans `data_phase2`

Objectif de ce bloc :

- inspecter le contenu réel de `sunspots.zip` (sans supposer à l’avance les noms de fichiers) ;
- extraire les fichiers **CSV** dans un sous-dossier dédié de la Phase 2 :

  `data_phase2/sunspots_raw/`

Contraintes :

- utiliser uniquement le zip local `sunspots.zip` déjà détecté en Bloc 1.0 ;
- ne jamais écrire en dehors de `SunspotPhase2Tlog` ;
- ne pas inventer de noms de fichiers : on se base sur les noms réellement présents dans le zip.

La cellule de code suivante va :

1. créer le dossier `data_phase2/sunspots_raw/` (s’il n’existe pas déjà) ;
2. ouvrir `sunspots.zip` et afficher la liste de tous ses fichiers ;
3. filtrer les fichiers dont le nom se termine par `.csv` (insensible à la casse) ;
4. extraire ces fichiers CSV sous `data_phase2/sunspots_raw/` ;
5. logguer le nombre de fichiers CSV trouvés et leur liste.

In [5]:
# Bloc 1.1 – Inspection et extraction de sunspots.zip

import zipfile
from pathlib import Path

# 1.1.1 – Dossier cible pour les données brutes Phase 2
DATA_PHASE2_RAW_DIR = PHASE2_ROOT / "data_phase2" / "sunspots_raw"
DATA_PHASE2_RAW_DIR.mkdir(parents=True, exist_ok=True)

print(f"Dossier cible pour les données extraites : {DATA_PHASE2_RAW_DIR}")

# 1.1.2 – Ouverture du zip et inspection du contenu
with zipfile.ZipFile(SUNSPOTS_ZIP_PATH, "r") as zf:
    members = zf.namelist()

print("\nContenu de sunspots.zip :")
for m in members:
    print(f"  - {m}")

# 1.1.3 – Filtrage des fichiers CSV (sans faire d'hypothèse sur les noms)
csv_members = [m for m in members if m.lower().endswith(".csv")]

if not csv_members:
    raise RuntimeError(
        "Aucun fichier .csv trouvé dans sunspots.zip. "
        "Vérifiez le contenu du zip Sunspots."
    )

print("\nFichiers CSV détectés dans le zip :")
for m in csv_members:
    print(f"  - {m}")

# 1.1.4 – Extraction des CSV dans data_phase2/sunspots_raw/
extracted_paths = []

with zipfile.ZipFile(SUNSPOTS_ZIP_PATH, "r") as zf:
    for m in csv_members:
        # On conserve la structure interne du zip sous data_phase2/sunspots_raw/
        target_path = DATA_PHASE2_RAW_DIR / m
        target_path.parent.mkdir(parents=True, exist_ok=True)
        with zf.open(m, "r") as src, open(target_path, "wb") as dst:
            dst.write(src.read())
        extracted_paths.append(str(target_path))

print("\nFichiers CSV extraits :")
for p in extracted_paths:
    print(f"  - {p}")

# 1.1.5 – Logging pour audit
log_message(
    "INFO",
    f"{len(extracted_paths)} fichier(s) CSV extrait(s) depuis sunspots.zip vers data_phase2/sunspots_raw/",
    block="BLOC_1.1",
)
log_metric(
    "sunspots_csv_extracted_count",
    len(extracted_paths),
    extra={"csv_files": extracted_paths},
)

Dossier cible pour les données extraites : C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\sunspots_raw

Contenu de sunspots.zip :
  - Sunspots.csv

Fichiers CSV détectés dans le zip :
  - Sunspots.csv

Fichiers CSV extraits :
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\sunspots_raw\Sunspots.csv
[STEP=3][INFO][BLOC_1.1] 1 fichier(s) CSV extrait(s) depuis sunspots.zip vers data_phase2/sunspots_raw/
[METRIC][sunspots_csv_extracted_count] = 1 (step=3)
